In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize


In [2]:

# IO flows in 1977
q0 = pd.read_csv('../../data/io1977.csv')
q0.index = q0.columns # index and columns are the same

q = pd.read_csv('../../data/io1982.csv')
q.index = q.columns


In [3]:

# Convert data and prior flows to coefficients/probabilities

# row totals of data
Y = q.sum(1)
# column totals of data
X = q.sum(0)


In [4]:

# prior coefficients, columns add to 1
q1 = q0 / q0.sum(1)

q0_uniform = np.ones_like(q1) / len(q1)


In [5]:
# Python: Modified entropy function
def entropy(prob, prior):
    epsilon = 1e-12
    return np.sum(prob * np.log((prob + epsilon) / (prior + epsilon)))


In [6]:
def classical_ce(a0, prob0, X, Y):
    """
    The pure Cross Entropy case
    """
    k = a0.shape[0]
    
    def objective(params):
        a = np.array(params).reshape((k, k))
        return entropy(a, prob0)
    
    def constraint_row(params):
        a = np.array(params).reshape((k, k))
        return np.dot(a, X) - Y  # Sum of products of coefficients and column totals equals row totals

    def constraint_col(params):
        a = np.array(params).reshape((k, k))
        return np.sum(a, axis=0) - 1  # Sum of coefficients in each column equals 1

    
    constraints = [
        {'type': 'eq', 'fun': constraint_row},
        {'type': 'eq', 'fun': constraint_col}
    ]

    bounds = [(0, 1) for _ in range(k*k)]
    
    result = minimize(objective, a0.flatten(), method='SLSQP', 
                      bounds=bounds, constraints=constraints, 
                      options={'ftol': 1e-12, 'maxiter': 1000})
    final_a = result.x.reshape((k, k))    
    return final_a, result

In [7]:

# Initialize
a_init = np.ones((11, 11)) / 11
uniform = a_init.copy()

# Solve
final_a, result = classical_ce(a_init, q0_uniform, X, Y)


In [8]:

k = q.shape[0]

final_a = pd.DataFrame(final_a)
final_a.rows = q.columns
final_a.columns = q.columns
# Print results
print(result)
print(final_a)
# print("Number of iterations:", len(monitor))

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 3.961099906704762
       x: [ 8.146e-02  6.224e-02 ...  4.201e-01  4.242e-01]
     nit: 625
     jac: [ 8.903e-01  6.211e-01 ...  2.531e+00  2.540e+00]
    nfev: 79684
    njev: 625
       FOODG     FEEDG   OTHERAG  RESOURCE  CONSTRUCT      COWS   FOODMFG  \
0   0.081465  0.062236  0.059057  0.001174   0.000801  0.041289  0.004967   
1   0.087819  0.080495  0.078462  0.016540   0.014115  0.069758  0.030356   
2   0.088246  0.081761  0.080197  0.019471   0.016797  0.072037  0.033935   
3   0.093051  0.096954  0.099734  0.112843   0.112154  0.101986  0.113060   
4   0.093253  0.097232  0.101347  0.115721   0.114984  0.102532  0.114867   
5   0.089954  0.087119  0.086529  0.037389   0.033974  0.081953  0.053039   
6   0.092350  0.095595  0.095747  0.097534   0.095987  0.099162  0.102291   
7   0.093152  0.098378  0.098793  0.130379   0.131291  0.105002  0.124645   
8   0.091762  0.098489  0.085097  0.13188

/var/folders/v2/2g59sj7s7qbcgl_0l5zkx1gm0000gn/T/ipykernel_85972/864896581.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  final_a.rows = q.columns


In [9]:
print(q1.values)

[[6.03304286e-02 1.10493709e-05 1.11440055e-05 7.60103711e-06
  1.21263342e-05 2.17782868e-03 1.22097066e-02 1.08124666e-05
  9.07689870e-06 3.43700917e-05 1.40347579e-03]
 [2.35957843e-05 3.25700885e-02 2.72493037e-05 1.79876878e-05
  2.47406877e-05 2.60747715e-01 1.78832845e-02 8.74136342e-05
  2.04297942e-05 5.21270718e-04 3.10493619e-03]
 [1.01834437e-04 6.53305630e-03 3.44293091e-02 4.05899682e-04
  3.11195163e-04 4.62381360e-04 5.29159362e-02 9.70215766e-04
  8.54241367e-03 6.51164867e-04 8.38602303e-03]
 [6.98969402e-02 6.35778605e-02 3.39234782e-02 4.08454208e-01
  3.63948099e-02 1.04995018e-02 1.30983208e-02 5.06921687e-02
  1.05766583e-02 2.69554923e-02 4.05293009e-03]
 [1.17624098e-02 1.17519320e-02 9.13447842e-03 2.22700244e-02
  1.14816947e-03 6.23232275e-03 4.56976504e-03 8.91503554e-03
  4.09961907e-03 2.38369658e-02 1.03927370e-01]
 [2.21328102e-02 3.64690514e-02 8.75653320e-03 7.52589776e-05
  6.14469281e-05 1.70287774e-01 1.94217378e-01 1.74514211e-04
  3.83421575e-04